In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils import get_model_data

In [ ]:
processed_path = '../data/processed/time_series_all.csv'
time_series_all = pd.read_csv(processed_path)

In [ ]:
df = get_model_data(time_series_all,date_range=(0,7), pred_day=7)
df.head()